# Dashboard

El dashboard no es más que un Consumer de Kafka que leerá del topic que hemos generado con KSQL. Hemos de añadir lógica que nos permita detectar cuándo se ha cerrado una ventana: Kafka empieza a generar mensajes en el topic de salida en cuanto empieza una ventana, incluso aunque luego tenga que actualizar un valor agregado porque hayan aparecido nuevos mensajes en el topic de entrada. Para evitar mostrar datos *intermedios* hemos optado por almacenar los datos de una ventana, actualizando los datos con cada nuevo mensaje, y sólo imprimiendo la ventana cuando ya hemos entrado en una nueva.

Nuestra aplicación no es muy sensible a los tiempos en los que aparecen los mensajes, así que un dashboard sencillo como este podría ser necesario. Si estuviéramos trabajando con datos de IOT o con registros en tiempo real de verdad habría que tener en cuenta que KSQL puede **actualizar** una ventana antigua con mensajes que hayan llegado *tarde* (por ejemplo, porque el broker que ha procesado ese mensaje ha tenido un pico de carga).


In [ ]:
from confluent_kafka import Consumer, KafkaError
import time
import json

from ejercicios.houses import TOPIC_DASHBOARD, DASHBOARD_GROUP

In [ ]:
# función auxiliar para imprimir de forma legible los datos de una ventana
# Cada ventana tiene el siguiente formato:
#     {
#         '1980': {'NUM_HOUSES': 2, 'AVG_PRICE': 10000.0},
#         '1990': {'NUM_HOUSES': 3, 'AVG_PRICE': 20000.0},
#         '2000': {'NUM_HOUSES': 1, 'AVG_PRICE': 50000.0},
#     }

def print_window(window):
    years = sorted(window.keys())
    for year in years:
        year_data = window[year]
        print("{}: {} houses at {:.2f}$".format(year, year_data['NUM_HOUSES'], year_data['AVG_PRICE']))

In [ ]:
c = Consumer({
    'bootstrap.servers': 'localhost:9092',
    'group.id': DASHBOARD_GROUP,
    'auto.offset.reset': 'earliest'
})
c.subscribe([TOPIC_DASHBOARD])

LATEST_WINDOW_END = 0
WINDOWS = {}


while True:
    msg = c.poll(1.0)


    if msg is None:
        continue

    if msg.error():
        print("Consumer error: {}".format(msg.error()))
        continue

    #print(msg.value())
    entry = json.loads(msg.value())
    if entry['WINDOW_END'] not in WINDOWS:
        WINDOWS[entry['WINDOW_END']] = {}  # inicializamos una ventana

    # Escribimos los estadísticos de una década en la ventana actual. Si ya hemos recibido
    # datos de esa década para este ventana con anterioridad, lo sobreescribimos.
    WINDOWS[entry['WINDOW_END']][entry['DECADE_BUILT']] = {'AVG_PRICE': entry['AVG_PRICE'], 'NUM_HOUSES': entry['NUM_HOUSES']}


    # Cambio de ventana: el mensaje tiene un tiempo de inicio de ventana posterior al final de ventana
    # más avanzado que hemos visto hasta ahora
    if entry['WINDOW_START'] >= LATEST_WINDOW_END and LATEST_WINDOW_END in WINDOWS:
        print_window(WINDOWS[LATEST_WINDOW_END])

    # Guardamos el final de ventana más avanzado que hemos visto hasta ahora
    LATEST_WINDOW_END = entry['WINDOW_END']
    print("*"*20)

c.close()